# `gu_toolkit` overview (updated)

This notebook is a hands-on tour of the current API surface.

It is organized so you can **test behavior directly**:
- each section says **what to do**,
- and what you should **expect to happen**.


In [ ]:
# Setup
from pathlib import Path
import sys

from pathlib import Path
import sys

# Walk up from current directory until we find a package root (has __init__.py)
pkg_root = Path.cwd().resolve()
while pkg_root != pkg_root.parent and not (pkg_root / "__init__.py").exists():
    pkg_root = pkg_root.parent
sys.path.insert(0, str(pkg_root.parent))

from gu_toolkit import *


## 1) Discoverability and introspection

**Do this:** run the next cells.

**Expect:**
- a quick map of major notebook-facing names,
- style options for `plot`,
- signatures/docstrings you can inspect interactively.


In [ ]:
# Top-level exported names (sample)
public_names = [name for name in dir() if not name.startswith("_")]
[name for name in public_names if name in {
    "Figure", "plot", "parameter", "params", "plots", "render",
    "numpify", "parse_latex", "NIntegrate", "NReal_Fourier_Series", "play"
}]


In [ ]:
plot_style_options()


In [ ]:
import inspect

print(inspect.signature(Figure.plot))
print(inspect.signature(numpify))
print(inspect.signature(NIntegrate))


## 2) Core plotting workflow (context-managed)

**Do this:** execute the next cell, then pan/zoom.

**Expect:**
- multiple traces appear on one figure,
- updating an existing `id` replaces that trace rather than creating a duplicate,
- re-renders happen as you change view ranges.


In [ ]:
fig1 = Figure(x_range=(-6, 6), y_range=(-2.5, 2.5), sampling_points=500)
display(fig1)

with fig1:
    set_title("Trigonometric overlays")
    plot(x, sin(x), id="sin")
    plot(x, cos(x), id="cos", dash="dash", color="#d62728")
    plot(x, sin(2*x), id="sin2", opacity=0.4, thickness=4)


In [ ]:
# Reusing id="sin" updates that trace.
with fig1:
    plot(x, sin(x**2/3), id="sin", color="#1f77b4", thickness=2)


## 3) Parameter sliders (automatic and explicit)

**Do this:** run the cells and move sliders in the right panel.

**Expect:**
- parameters are auto-discovered from free symbols,
- manual slider registration works too,
- figure updates immediately when sliders move.


In [ ]:
fig2 = Figure(x_range=(-8, 8), y_range=(-4, 4))
display(fig2)

with fig2:
    set_title("Auto-parameterized expression")
    plot(x, a*sin(b*x + c), id="wave")


In [ ]:
# Explicit slider defaults and ranges
with fig2:
    parameter(a, min=-3, max=3, value=1.0, step=0.05, readout_format=".2f")
    parameter(b, min=0.2, max=4.0, value=1.0, step=0.05)
    parameter(c, min=-3.14, max=3.14, value=0.0, step=0.01)


In [ ]:
# Current numeric parameter values
with fig2:
    params_snapshot = params.snapshot()
params_snapshot

## 4) Plot/figure introspection

**Do this:** inspect figure and plot objects.

**Expect:**
- discoverable plot ids,
- access to symbolic expression and live numeric expression,
- access to last sampled x/y arrays for debugging/analysis.


In [ ]:
list(fig2.plots.keys())


In [ ]:
wave = fig2.plots["wave"]
wave.symbolic_expression, wave.parameters


In [ ]:
# Numeric callable (parameter-aware through the figure context)
wave_numeric = wave.numeric_expression
wave_numeric(np.array([0.0, 1.0, 2.0]))


In [ ]:
# Last sampled arrays used for rendering (None before first render)
wave.x_data[:5], wave.y_data[:5]


## 5) Freezing, unfreezing, and parameter snapshots

**Do this:** run the cell and compare the outputs.

**Expect:**
- a snapshot captures current slider values,
- `freeze(...)` freezes numeric evaluation at chosen values,
- `unfreeze()` restores explicit function arguments for unfrozen parameters (pass current values each call).


In [ ]:
f_live = wave.numeric_expression

with wave.figure:
    snap = params.snapshot(full=True)
    
f_frozen = f_live.freeze(snap.value_map())

print("live @ x=1:", f_live(1.0))
print("frozen @ x=1:", f_frozen(1.0))

with wave.figure:
    params[a] = 2.0
    
print("after changing a...")
print("live @ x=1:", f_live(1.0))
print("frozen @ x=1 (unchanged):", f_frozen(1.0))

f_frozen = f_frozen.unfreeze()
with wave.figure:
    a_val = params[a].value
    b_val = params[b].value
    c_val = params[c].value
print("unfrozen now tracks call-time arguments again:", f_frozen(1.0, a_val, b_val, c_val))

with wave.figure:
    params[a] = 3.0
    a_val = params[a].value
    b_val = params[b].value
    c_val = params[c].value
print("after another slider update, pass refreshed values:", f_frozen(1.0, a_val, b_val, c_val))


## 6) `numpify` directly (without plotting)

**Do this:** compile a symbolic expression and evaluate it as a NumPy-aware callable.

**Expect:**
- vectorized evaluation,
- optional freezing/binding workflows independent of figures.


In [ ]:
f = numpify(a*exp(-x**2) * cos(b*x), vars=(x, a, b))
f(np.linspace(-2, 2, 5), 1.0, 2.0)


In [ ]:
f_bound = f.freeze({a: 1.5, b: 3.0})
f_bound(np.linspace(-2, 2, 5))


You can use `numpify` to numerically evaluate constants. However `numpify` returns functions so do not forget to "call" the numpified expression

In [ ]:
numpify(pi**(1/2))()

## 7) Parse LaTeX into SymPy, then plot

**Do this:** parse, inspect, and plot.

**Expect:**
- LaTeX turns into a SymPy expression,
- expression can be plotted like any other symbolic input.


In [ ]:
expr = parse_latex(r"\sin(x) + \frac{1}{2}\cos(3x)")
expr


In [ ]:
fig3 = Figure(x_range=(-6, 6), y_range=(-2, 2))
display(fig3)
with fig3:
    set_title("Expression parsed from LaTeX")
    plot(x, expr, id="latex_expr", color="#2ca02c")

#BUG HERE:  AttributeError: 'Tree' object has no attribute 'free_symbols'
#PROBLEM: the parser seturns a Tree and it is ambiguious. This is bullshit The expression $\sin(x) + \frac{1}{2}\cos(3x)$ is absolutely unambiguous
#Tree('_ambig', [sin(x + cos(3*x)/2), sin(x + 1/2)*cos(3*x), sin(x) + cos(3*x)/2])

## 8) Numeric helpers from the prelude

### 8.1 `NIntegrate`

**Do this:** integrate a symbolic expression.

**Expect:**
- a scalar float close to known analytic values.


In [ ]:
print(
    numpify(pi**(1/2))()
)
print(
    NIntegrate(exp(-x**2), (x, -oo, oo))  # should be close to sqrt(pi)
     )

### 8.2 `NReal_Fourier_Series`

**Do this:** compute Fourier coefficients on a finite interval.

**Expect:**
- two arrays `(cos_coeffs, sin_coeffs)`; index 0 is the DC component.


In [ ]:
cos_coeffs, sin_coeffs = NReal_Fourier_Series(sin(x) + 0.3*cos(2*x), (x, -pi, pi), samples=2048)
cos_coeffs[:6], sin_coeffs[:6]


### 8.3 `play`

**Do this:** evaluate the cell.

**Expect:**
- an audio widget appears and autoplays,
- use `loop=False` for one-shot playback.


In [ ]:
play(sin(2*pi*220*x) * exp(-2*x), (x, 0, 1.5), loop=False)
#BUG, autoplays without display! Should not do by default unless autoplay=True)


## 9) Common examples you can adapt quickly

These are compact patterns you can copy for day-to-day use.


In [ ]:
# Example A: Compare two models with one shared parameter
figA = Figure(x_range=(-4, 4), y_range=(-2, 2))
display(figA)
with figA:
    plot(x, tanh(k*x), id="tanh", label="tanh(kx)")
    plot(x, x/sqrt(1 + (k*x)**2), id="softsign", label="softsign-like", dash="dot")
    parameter(k, min=0.1, max=5, value=1.0, step=0.1)


In [ ]:
with figA:
    print([k for k in plots.keys()])

In [ ]:
# Example B: Build expressions with indexed symbol families
expr_family = a[1]*sin(x) + a[2]*cos(2*x) + a[3]*sin(3*x)
expr_family


In [ ]:
figB = Figure(x_range=(-6, 6), y_range=(-4, 4))
display(figB)
with figB:
    plot(x, expr_family, id="family")
    parameter(a[1], min=-2, max=2, value=1)
    parameter(a[2], min=-2, max=2, value=0.5)
    parameter(a[3], min=-2, max=2, value=0.2)


## 10) What to explore next

- Use `help(Figure)` and `help(Figure.plot)` for full docstrings.
- Inspect `fig.plots[<id>]` objects to understand data flow from symbolic to numeric.
- Explore the `tests/` and `documentation/develop_guide/` folders for deeper patterns.
